In [1]:
import glob
import time
import pandas as pd
# from xml.dom import minidom
from lxml import etree
from nltk import ngrams
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
%matplotlib inline

In [3]:

t1=time.time()
path='./dataset/blogs_train/'
all_files = glob.glob(path + "/*.xml")
li = []

for filename in all_files:
  # print("file name ", filename)
  li.append(filename)
    

t2=time.time()
print("time taken to read All csv :: ", t2-t1, " seconds ")

time taken to read All csv ::  0.03694343566894531  seconds 


In [4]:
def clean_file_names(li):
  # names_list=[]
  # for i in range(len(li)):
  name_list=li.split('/')[-1]
  return name_list

In [5]:
def get_attributes_from_name(name):
  # data_object=[]
  # for data in range(len(names_list)):
  #   dummy_var=names_list[data].split('.')
  #   dummy_object={'id':dummy_var[0], 'gender':dummy_var[1],
  #                 'age':dummy_var[2], 'category':dummy_var[3],
  #                 'horoscope':dummy_var[4]
  #                 }
  #   data_object.append(dummy_object)
  dummy_var=name.split('.')
  data_object={'id':dummy_var[0], 'gender':dummy_var[1],
                  'age':dummy_var[2], 'category':dummy_var[3],
                  'horoscope':dummy_var[4]
                }
  
  return data_object

In [6]:
parser = etree.XMLParser(encoding='ISO-8859-1', recover = True, huge_tree=True)

In [7]:
def read_xml(li):
  all_objects=[]
  for i in range(len(li)):
    # print("i will run ", i+1," times")
    name_of_file=clean_file_names(li[i])
    # print(name_of_file)
    data_object=get_attributes_from_name(name_of_file)
    # print(data_object)

    post_text='  '
    tree = etree.parse(li[i], parser=parser)
    root = tree.getroot()
    for element in root.iter():
        if element.tag == 'post':
            element_text = element.text
            post_text = post_text + element_text
    mystring = post_text.replace('\n', '').replace('\r', '').replace('\t', '')
    mystring = mystring.replace('   ', '').replace('\\','')
   


   
    # print(mystring)
    # name_of_file=clean_file_names(li[i])
    # data_object=get_attributes_from_name(name_of_file)
    



    data_object['posts']=mystring   
    all_objects.append(data_object)

  return all_objects



In [8]:
len(li)

14600

In [9]:
t1=time.time()
data=read_xml(li[0:])
t2=time.time()

In [10]:
print("time taken to read :: ", (t2-t1)/100)

time taken to read ::  1.095266056060791


In [11]:
df=pd.DataFrame(data)

In [12]:
df.tail()

,id,gender,age,category,horoscope,posts
14595,3122872,male,15,Student,Libra,ok we got webspace off abp for bgs but of cour...
14596,3322392,female,15,indUnk,Pisces,"dearest diary, i do not know wad to put for t..."
14597,4008208,female,16,indUnk,Pisces,Good friends are the greatest treasures of th...
14598,1599685,male,23,Student,Capricorn,WE BRING THIS SPECIAL ANNOUCEMENT TO YOU LIVE...
14599,1608709,male,25,Accounting,Aries,Football Alex 'Knight of the Realm' Ferguson ...


In [13]:
data=''

## Multilabel classification

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [16]:
from sklearn.feature_extraction.text import CountVectorizer


In [17]:
sentences = df['posts'].values

In [18]:
labels= []
for hor,gen in zip(df['horoscope'],df['gender']):
    labels.append((hor,gen))

In [19]:
mlb = MultiLabelBinarizer()
mlb.fit(labels)
classes=len(mlb.classes_)
print("classes  :: ", classes)
mlb.classes_

classes  ::  14


array(['Aquarius', 'Aries', 'Cancer', 'Capricorn', 'Gemini', 'Leo',
       'Libra', 'Pisces', 'Sagittarius', 'Scorpio', 'Taurus', 'Virgo',
       'female', 'male'], dtype=object)

In [20]:
labels = mlb.fit_transform(labels)

In [21]:
y=labels

In [22]:
(trainX, testX, trainY, testY) = train_test_split(sentences,y, test_size=0.2, random_state=42)

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(trainX)

X_train = vectorizer.transform(trainX)
X_test  = vectorizer.transform(testX)

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
# model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes, activation='softmax'))
# model.add(Activation('softmax'))


In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

In [ ]:
classifier_nn = model.fit(X_train,trainY,
                    epochs=5,
                    verbose=True,
                    validation_data=(X_test, testY),
                    batch_size=10)

In [84]:
model.predict(X_test[4])

array([[0.04229993, 0.01288046, 0.01775206, 0.01131918, 0.02503961,
        0.02156646, 0.01341454, 0.02311417, 0.0022669 , 0.04635284,
        0.01827584, 0.00922095, 0.7274192 , 0.02907783]], dtype=float32)

In [85]:
testY[4]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [86]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
# model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes, activation='sigmoid'))
# model.add(Activation('softmax'))


In [87]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 20)                13935920  
_________________________________________________________________
dense_61 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_62 (Dense)             (None, 1024)              11264     
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_23 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_63 (Dense)             (None, 14)                14350     
Total params: 13,965,840
Trainable params: 13,963,792
Non-trainable params: 2,048
____________________________________________________________

In [88]:
classifier_nn = model.fit(X_train,trainY,
                    epochs=5,
                    verbose=True,
                    validation_data=(X_test, testY),
                    batch_size=10)

Train on 11680 samples, validate on 2920 samples
Epoch 1/5
11680/11680 [==============================] - 215s 18ms/step - loss: 0.3590 - acc: 0.8523 - val_loss: 0.5803 - val_acc: 0.7981
Epoch 2/5
11680/11680 [==============================] - 214s 18ms/step - loss: 0.3475 - acc: 0.8571 - val_loss: 5.5809 - val_acc: 0.1411
Epoch 3/5
11680/11680 [==============================] - 207s 18ms/step - loss: 0.3480 - acc: 0.8570 - val_loss: 0.8345 - val_acc: 0.8553
Epoch 4/5
11680/11680 [==============================] - 204s 17ms/step - loss: 0.3482 - acc: 0.8558 - val_loss: 0.9040 - val_acc: 0.6826
Epoch 5/5
11680/11680 [==============================] - 204s 17ms/step - loss: 0.3467 - acc: 0.8582 - val_loss: 1.0090 - val_acc: 0.4386


In [89]:
model.predict(X_test[4])

array([[0.1749047 , 0.20853055, 0.6293377 , 0.7079589 , 0.52126914,
        0.628846  , 0.02096906, 0.19228855, 0.5503968 , 0.79130846,
        0.45031193, 0.9912372 , 0.8133841 , 0.22002329]], dtype=float32)

In [90]:
testY[4]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])